In [85]:
# run nvidia-smi to check for available gpus, and change cuda_device_num value below
import torch
cuda_device_num = 1
device = torch.device("cuda:" + str(cuda_device_num) if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.set_device(cuda_device_num)
torch.cuda.current_device()

cuda:1


1

In [86]:
# modify these paths to your system's
MODEL_METHOD_MUTATION_PATH = "/home/chenghin/Desktop/repos/java-mutation-framework/models/code-generation/saved_models/checkpoint-best-score"
MODEL_COMMENT_MUTATION_PATH = "/home/chenghin/Desktop/repos/java-mutation-framework/models/codet5_base_all_lr5_bs32_src64_trg64_pat5_e10/checkpoint-best-bleu"

In [87]:
from model import Seq2Seq # Copy pasted from https://github.com/microsoft/CodeBERT/blob/master/UniXcoder/downstream-tasks/code-generation/model.py
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, T5ForConditionalGeneration, AutoTokenizer

PRETRAINED_MODEL_NAME_UNIXCODER = "microsoft/unixcoder-base"
PRETRAINED_MODEL_NAME_CODET5 = "Salesforce/codet5-base-multi-sum"

class TokenizerModelPair:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

def build_model(pretrained_model_name):
    if pretrained_model_name == PRETRAINED_MODEL_NAME_UNIXCODER:
        # build model
        tokenizer = RobertaTokenizer.from_pretrained(pretrained_model_name)
        config = RobertaConfig.from_pretrained(pretrained_model_name)
        # important！You must set is_decoder to True for generation
        config.is_decoder = True
        encoder = RobertaModel.from_pretrained(pretrained_model_name,config=config)
        model = Seq2Seq(encoder=encoder,decoder=encoder,config=config,
                        beam_size=10,max_length=256,
                        sos_id=tokenizer.convert_tokens_to_ids(["<mask0>"])[0],eos_id=tokenizer.sep_token_id)
    else:
        # build model
        tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
        model = T5ForConditionalGeneration.from_pretrained(pretrained_model_name)
    return TokenizerModelPair(model, tokenizer)

In [88]:
unixcoder_model_and_tokenizer = build_model(PRETRAINED_MODEL_NAME_UNIXCODER)
codet5_model_and_tokenizer = build_model(PRETRAINED_MODEL_NAME_CODET5)

Updated!!!


In [89]:
import os
def load_model(model, load_model_path='fine_tuned_models'):
    model_to_load = model.module if hasattr(model, 'module') else model
    load_model_path = os.path.join(load_model_path, 'pytorch_model.bin')
    model.load_state_dict(torch.load(load_model_path, map_location='cpu'))
    model.to(device)

In [90]:
load_model(unixcoder_model_and_tokenizer.model, MODEL_METHOD_MUTATION_PATH)
load_model(codet5_model_and_tokenizer.model, MODEL_COMMENT_MUTATION_PATH)

In [91]:
# def get_tokens

def predict(model, tokenizer, code, gold):
    input_ids = tokenizer(code, return_tensors="pt").input_ids
    generated_ids = model.generate(input_ids.to(device))
    if len(generated_ids.size()) == 3: # decoder takes in a 2d tensor. If 3d, remove the outer dimension
        generated_ids = torch.squeeze(generated_ids, 0)
    comment = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return comment, getSmoothBLEU4(gold, comment)

In [92]:
import nltk
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
def getSmoothBLEU4(gold, result):
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([gold.split()], result.split(), weights = [0.25,0.25,0.25,0.25], smoothing_function=chencherry.method2)
    return round(BLEUscore,4)

In [93]:
code = '''
public RequestMethodsRequestCondition getMatchingCondition(ServerWebExchange exchange) {
                if (CorsUtils.isPreFlightRequest(exchange.getRequest())) {
                        return matchPreFlight(exchange.getRequest());
                }
                if (getMethods().isEmpty()) {
                        if (RequestMethod.OPTIONS.name().equals(exchange.getRequest().getMethodValue())) {
                                return null; // We handle OPTIONS transparently, so don't match if no explicit declarations
                        }
                        return this;
                }
                return matchRequestMethod(exchange.getRequest().getMethod());
        }

'''

In [94]:
gold = 'check if any of the http request methods match the given request and return an instance that contains the matching http request method only'

In [95]:
predict(unixcoder_model_and_tokenizer.model, unixcoder_model_and_tokenizer.tokenizer, code, gold) # test method mutation

("public RequestMethodsRequestCondition getMatchingCondition(ServerWebExchange exchange) {\n                if (CorsUtils.isPreFlightRequest(exchange.getRequest())) {\n                        return matchPreFlight(exchange.getRequest());\n                }\n                if (getMethods().isEmpty()) {\n                        if (RequestMethod.OPTIONS.name().equals(exchange.getRequest().getMethodValue())) {\n                                return null; // We handle OPTIONS transparently, so don't match if no explicit declarations\n                        }\n                        return this;\n                }",
 0.0384)

In [96]:
predict(codet5_model_and_tokenizer.model, codet5_model_and_tokenizer.tokenizer, gold, code) # test comment mutation

/home/chenghin/anaconda3/envs/explainable-mutation/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


('if any of the http request methods match the given request and return an instance that contains',
 0.0236)

In [97]:
class ExplainableMutator:
    def __init__(self, comment_mutation_model, comment_mutation_tokenizer,
                 method_mutation_model, method_mutation_tokenizer):
        self.comment_mutation_model = comment_mutation_model
        self.comment_mutation_tokenizer = comment_mutation_tokenizer
        self.method_mutation_model = method_mutation_model
        self.method_mutation_tokenizer = method_mutation_tokenizer

    def generate(self, comment, method_body):
        mutated_comment = predict(self.comment_mutation_model, self.comment_mutation_tokenizer, 
                                  comment, comment)[0]
        method_with_comment = comment + "</s>" + method_body
        mutated_method = predict(self.method_mutation_model, self.method_mutation_tokenizer, 
                                 method_with_comment, method_with_comment)[0]
        return [mutated_comment, mutated_method]

In [ ]:
# Tests
import unittest
from numpy.testing import assert_array_equal

TEST_FILES_PATH = "./test-files"

class TestExplainableMutator(unittest.TestCase):
    
    def setUp(self):
        self.mutator = ExplainableMutator(codet5_model_and_tokenizer.model, codet5_model_and_tokenizer.tokenizer,
                                         unixcoder_model_and_tokenizer.model, unixcoder_model_and_tokenizer.tokenizer) 

# ======================== Integration tests =====================
    def test_generate(self):
        comment = "does nothing."
        method_body = """public void method() {
            int a = 0;
            return;
        }"""
        [mutated_comment, mutated_method] = self.mutator.generate(comment, method_body)
        print(mutated_comment)
        print(mutated_method)
        self.assertNotEqual(comment, mutated_comment)
        self.assertNotEqual(method_body, mutated_method)

        
unittest.main(argv=[''], verbosity=2, exit=False)

In [99]:
%run ./SocketServer.ipynb

In [100]:
class ExplainableMutationSocketServer(SocketServer):
    def __init__(self, host, port, explainable_mutator):
        super().__init__(host, port)
        self.explainable_mutator = explainable_mutator

    def func(self):
        while True:
            print("-"*20)
            comment = self.recvMsg()
            method_body = self.recvMsg()
            try:
                [mutated_comment, mutated_method] = self.explainable_mutator.generate(comment, method_body)
            except RuntimeError as e:
                self.sendMsg("FAIL")
                self.sendMsg(str(e))
                continue
            self.sendMsg(mutated_comment)
            self.sendMsg(mutated_method)

In [101]:
HOST = "127.0.0.1"
PORT = 8080
server = ExplainableMutationSocketServer(HOST, PORT, ExplainableMutator(codet5_model_and_tokenizer.model, codet5_model_and_tokenizer.tokenizer,
                                         unixcoder_model_and_tokenizer.model, unixcoder_model_and_tokenizer.tokenizer) )

In [ ]:
server.start()

Connected by ('127.0.0.1', 33766)
--------------------
request: Sets the weight for the plot and sends a   to all registered listeners.   the weight.
request: public void setWeight(int weight){
  this.weight=weight;
  fireChangeEvent();
}

response: b'Sets the weight for the plot and sends a   to all registered listeners.   the weight'
response: b'public void setWeight(int weight){\n  this.weight=weight;\n  fireChangeEvent();\n}'
--------------------
request: Returns <code>true</code> if the domain gridlines are visible, and <code>false<code> otherwise.  <code>true</code> or <code>false</code>.
request: public boolean isDomainGridlinesVisible(){
  return this.domainGridlinesVisible;
}

response: b'This method returns <code>true</code> if the domain gridlines are visible,'
response: b'public boolean isDomainGridlinesVisible(){\n  return this.domainGridlinesVisible;\n}'
--------------------
request: Sets the flag that controls whether or not the domain grid-lines are visible. <p> If the 

response: b'Sets the flag that controls whether or not the range axis grid lines are visible. <p'
response: b'public void setRangeGridlinesVisible(boolean visible){\n  if (this.rangeGridlinesVisible!= visible) {\n    this.rangeGridlinesVisible=visible;\n    fireChangeEvent();\n  }'
--------------------
request: Returns the stroke for the grid lines (if any) plotted against the range axis.  The stroke (never <code>null</code>).
request: public Stroke getRangeGridlineStroke(){
  return this.rangeGridlineStroke;
}

response: b'The stroke for the grid lines (if any) plotted against the range axis. The'
response: b'public Stroke getRangeGridlineStroke(){\n  return this.rangeGridlineStroke;\n}'
--------------------
request: Sets the stroke for the grid lines plotted against the range axis, and sends a   to all registered listeners.   the stroke (<code>null</code> not permitted).
request: public void setRangeGridlineStroke(Stroke stroke){
  if (stroke == null) {
    throw new IllegalArgumentE

request: Returns the paint for the zero baseline (if any) plotted against the domain axis.  1.0.5  The paint (never <code>null</code>).
request: public Paint getDomainZeroBaselinePaint(){
  return this.domainZeroBaselinePaint;
}

response: b'1.0.5 The paint (never <code>null</code'
response: b'public Paint getDomainZeroBaselinePaint(){\n  return this.domainZeroBaselinePaint;\n}'
--------------------
request: Sets the paint for the zero baseline plotted against the domain axis and sends a   to all registered listeners.   the paint (<code>null</code> not permitted).  1.0.5
request: public void setDomainZeroBaselinePaint(Paint paint){
  if (paint == null) {
    throw new IllegalArgumentException("Null 'paint' argument.");
  }
  this.domainZeroBaselinePaint=paint;
  fireChangeEvent();
}

response: b'Sets the paint for the zero baseline plotted against the domain axis and sends a   to'
response: b'public void setDomainZeroBaselinePaint(Paint paint){\n  if (paint == null) {\n    throw new Il

response: b'Sets the paint used for the specified quadrant and sends a message to all registered listeners.'
response: b'public void setQuadrantPaint(int index,Paint paint){\n  if (index < 0 || index > 3) {\n    throw new IllegalArgumentException("The index value (" + index + ") should be in the range 0 to 3.");\n  }\n  this.quadrantPaint[index]=paint;\n  fireChangeEvent();\n}'
--------------------
request: Adds a marker for the domain axis and sends a  to all registered listeners. <P> Typically a marker will be drawn by the renderer as a line perpendicular to the range axis, however this is entirely up to the renderer.   the marker (<code>null</code> not permitted).
request: public void addDomainMarker(Marker marker){
  addDomainMarker(marker,Layer.FOREGROUND);
}

response: b'Adds a marker for the domain axis and sends a message to all registered listeners. <P'
response: b'public void addDomainMarker(Marker marker){\n  addDomainMarker(marker,Layer.FOREGROUND);\n}'
--------------------

request: Removes a marker for a specific dataset/renderer and sends a  to all registered listeners.  the dataset/renderer index.  the marker.  the layer (foreground or background).  A boolean indicating whether or not the marker was actually removed.  1.0.7
request: public boolean removeDomainMarker(int index,Marker marker,Layer layer){
  return removeDomainMarker(index,marker,layer,true);
}

response: b'This method removes a marker for a specific dataset/renderer and sends a  to all registered'
response: b'public boolean removeDomainMarker(int index,Marker marker,Layer layer){\n  return removeDomainMarker(index,marker,layer,false);\n}'
--------------------
request: Removes a marker for a specific dataset/renderer and, if requested, sends a   to all registered listeners.   the dataset/renderer index.   the marker.   the layer (foreground or background).   notify listeners?  A boolean indicating whether or not the marker was actually removed.  1.0.10
request: public boolean removeDomain

...

request: Tests this <code>GrayPaintScale</code> instance for equality with an arbitrary object.  This method returns <code>true</code> if and only if: <ul> <li><code>obj</code> is not <code>null</code>;</li> <li><code>obj</code> is an instance of <code>GrayPaintScale</code>;</li> </ul>   the object (<code>null</code> permitted).  A boolean.
request: public boolean equals(Object obj){
  if (obj == this) {
    return true;
  }
  if (!(obj instanceof GrayPaintScale)) {
    return false;
  }
  GrayPaintScale that=(GrayPaintScale)obj;
  if (this.lowerBound != that.lowerBound) {
    return false;
  }
  if (this.upperBound != that.upperBound) {
    return false;
  }
  return true;
}

response: b'This method tests this <code>GrayPaintScale</code> instance for equality with an'
response: b'public boolean equals(Object obj){ if (obj == null || obj instanceof GrayPaintScale) { return false; }\n  if (!(obj instanceof GrayPaintScale)) {\n    return true;\n  }'
--------------------
request: Returns 

response: b'FAIL'
response: b'The expanded size of the tensor (1180) must match the existing size (1026) at non-singleton dimension 1.  Target sizes: [1, 1180].  Tensor sizes: [1, 1026]'
--------------------
request: Draws an item for a plot with a vertical orientation.   the graphics device.   the renderer state.   the data area.   the plot.   the domain axis.   the range axis.   the data.   the row index (zero-based).   the column index (zero-based).
request: protected void drawVerticalItem(Graphics2D g2,CategoryItemRendererState state,Rectangle2D dataArea,CategoryPlot plot,CategoryAxis domainAxis,ValueAxis rangeAxis,StatisticalCategoryDataset dataset,int row,int column){
  RectangleEdge xAxisLocation=plot.getDomainAxisEdge();
  double rectX=domainAxis.getCategoryStart(column,getColumnCount(),dataArea,xAxisLocation);
  int seriesCount=getRowCount();
  int categoryCount=getColumnCount();
  if (seriesCount > 1) {
    double seriesGap=dataArea.getWidth() * getItemMargin() / (categoryCou

request: Returns <code>true</code> if the axis is visible, and  <code>false</code> otherwise.  A boolean.
request: public boolean isVisible(){
  return this.visible;
}

response: b'This method returns a boolean value indicating whether the axis is visible, and \xe2\x80\x93'
response: b'public boolean isVisible(){\n  return this.visible;\n}'
--------------------
request: Sets a flag that controls whether or not the axis is visible and sends  an   to all registered listeners.   the flag.
request: public void setVisible(boolean flag){
  if (flag != this.visible) {
    this.visible=flag;
    notifyListeners(new AxisChangeEvent(this));
  }
}

response: b'Sets a flag that controls whether or not the axis is visible and sends an   to'
response: b'public void setVisible(boolean flag){\n  if (flag!= this.visible) {\n    this.visible=flag;\n    notifyListeners(new AxisChangeEvent(this));\n  }'
--------------------
request: Returns the label for the axis.  The label for the axis (<code>null</code

response: b'A boolean that controls whether the axis line is drawn. A boolean.'
response: b'public boolean isAxisLineVisible(){\n  return this.axisLineVisible;\n}'
--------------------
request: Sets a flag that controls whether or not the axis line is visible and  sends an   to all registered listeners.   the flag.
request: public void setAxisLineVisible(boolean visible){
  this.axisLineVisible=visible;
  notifyListeners(new AxisChangeEvent(this));
}

response: b'Sets a flag that controls whether or not the axis line is visible and  sends an  '
response: b'public void setAxisLineVisible(boolean visible){\n  this.axisLineVisible=visible;\n  notifyListeners(new AxisChangeEvent(this));\n}'
--------------------
request: Returns the paint used to draw the axis line.  The paint (never <code>null</code>).
request: public Paint getAxisLinePaint(){
  return this.axisLinePaint;
}

response: b'This method returns the paint used to draw the axis line.'
response: b'public Paint getAxisLinePaint(){\

request: Returns the inside length of the tick marks.  The length.
request: public float getTickMarkInsideLength(){
  return this.tickMarkInsideLength;
}

response: b'This method returns the inside length of the tick marks.'
response: b'public float getTickMarkInsideLength(){\n  return this.tickMarkInsideLength;\n}'
--------------------
request: Sets the inside length of the tick marks and sends an   to all registered listeners.   the new length.
request: public void setTickMarkInsideLength(float length){
  this.tickMarkInsideLength=length;
  notifyListeners(new AxisChangeEvent(this));
}

response: b'Sets the inside length of the tick marks and sends an   to all registered listeners.  '
response: b'public void setTickMarkInsideLength(float length){\n  this.tickMarkInsideLength=length;\n  notifyListeners(new AxisChangeEvent(this));\n}'
--------------------
request: Returns the outside length of the tick marks.  The length.
request: public float getTickMarkOutsideLength(){
  return this.